This [Jupyter notebook](https://jupyter.org/)
is meant to be viewed in "slide mode".

If you want to view the slides locally,
you'll need to install [RISE](https://rise.readthedocs.io/).

But it's probably simpler to just visit the
[online version](https://nbviewer.jupyter.org/format/slides/github/mgeier/splines2021/blob/main/presentation.ipynb).

# Rotation Splines for Spatial Audio Scenes

Matthias Geier

https://github.com/mgeier/splines2021

## Audio Scene Description Format (ASDF)

* https://AudioSceneDescriptionFormat.readthedocs.io/
* https://github.com/AudioSceneDescriptionFormat/asdf-rust
* https://github.com/SoundScapeRenderer/ssr/pull/155
* splines for sound source trajectories
* splines for rotation
  * source rotations
  * rotation of groups of sources
  * rotation of local coordinate systems
* splines for volume envelopes
* everything is still *work in progress*!

## Overview

* splines in Euclidean space
  * Bézier splines
  * De Casteljau's algorithm
  * Catmull–Rom splines
* three-dimensional rotation
* unit quaternions
  * Slerp (**s**pherical **l**inear int**erp**olation)
* rotation splines
  * De Casteljau with Slerp
  * Catmull–Rom with Slerp

## Splines in Euclidean Space

* https://splines.readthedocs.io/
* https://splines.readthedocs.io/euclidean/

## Parametric Piecewise Cubic Polynomial Curves

* $\boldsymbol{p}_i(t) = \boldsymbol{d}_i t^3 + \boldsymbol{c}_i t^2 + \boldsymbol{b}_i t + \boldsymbol{a}_i$
  * univariate
  * $n$-dimensional
  * uniform vs. non-uniform

## Bézier Splines

* De Casteljau's algorithm
  * https://splines.readthedocs.io/euclidean/bezier-de-casteljau.html

## Catmull–Rom Splines

* https://splines.readthedocs.io/euclidean/catmull-rom-properties.html

## Three-dimensional Rotation

* azimuth, elevation, roll
  * order matters!
  * gimbal lock
* axis and angle
* rotation matrix
* unit quaternion
* ... and more: homogeneous coordinates, geometric algebra, ...

## Quaternions

* a + bi + cj + dk
  * $i^2 = \dots$
* scalar and vector
* 4D vector space

## Unit Quaternions

## Slerp

* https://splines.readthedocs.io/rotation/slerp.html

## Rotation Splines

* https://splines.readthedocs.io/rotation/de-casteljau.html
* https://splines.readthedocs.io/rotation/catmull-rom-uniform.html
* https://splines.readthedocs.io/rotation/catmull-rom-non-uniform.html